In [9]:
import torch # Import PyTorch 神经网络库
import cv2   #opencv 图像处理库
import ipywidgets #jupyter notebook的交互式小部件
from cnn import * #导入CNN模型
import numpy as np #导入numpy库
# import serial #导入串口库
import struct #导入struct库，打包数据使用
# 串口初始化
# ser = serial.Serial('/dev/ttyAMA0',115200)

cnn=CNN() # create CNN model，类的实例化
# 加载网络模型
cnn.load_state_dict(torch.load('weight/pytorchp.pkl',map_location='cpu'))

<All keys matched successfully>

In [10]:

def findRoi(frame, thresValue,kernel):
    rois = [] #存放数字区域
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  #转灰度图
    ret, gray = cv2.threshold(gray,thresValue,255,cv2.THRESH_BINARY_INV) #二值化
    gray = cv2.dilate(gray,kernel,iterations=4)  #膨胀

    gray = cv2.erode(gray,kernel,iterations=2)  #腐蚀
    
    gray = cv2.dilate(gray,kernel,iterations=4) #膨胀

    contours, hierarchy = cv2.findContours(gray,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE) #轮廓检测
    area = [] #存放轮廓面积
    # 找到最大的轮廓
    for k in range(len(contours)):
        area.append(cv2.contourArea(contours[k])) #计算轮廓面积
    if area  :  
        max_idx = np.argmax(np.array(area)) #找到最大面积的轮廓
        x, y, w, h = cv2.boundingRect(contours[max_idx]) #找到轮廓的外接矩形
        rois.append((x,y,w,h)) #存放数字区域
    return rois, gray,area
    
    
    
# 识别数字
def findDigit(cnn, roi, thresValue):
    ret, th = cv2.threshold(roi, thresValue, 255, cv2.THRESH_BINARY) #二值化
    th = cv2.resize(th,(28,28),interpolation=cv2.INTER_AREA) #resize
    newing=[[th]]   #增加一个维度
    newing = torch.Tensor(np.array(newing)).float()/255.    #归一化
    predicions=cnn(newing)  #预测
    retult=torch.argmax(predicions).detach().numpy()    #取最大值
    return retult,th #返回预测值和二值化图像


In [12]:
font_scale=1.5 #字体大小
font=cv2.FONT_HERSHEY_PLAIN #字体类型
kernel = np.ones((4, 4), np.uint8) #膨胀腐蚀核
wid = ipywidgets.Image() #创建一个图像显示窗口
# cap=cv2.VideoCapture(0) #打开摄像头
# cap=cv2.VideoCapture("2.mp4") #打开视频
# cap=cv2.VideoCapture("1.mp4") #打开视频
cap=cv2.VideoCapture("4.mp4") #打开视频
if not cap.isOpened(): #如果打开失败
    cap=cv2.VideoCapture(1) #打开其他摄像头
if not cap.isOpened(): #如果打开失败
    raise IOError('Can not open video') #抛出异常
cap.set(cv2.CAP_PROP_FPS, 30) #设置帧率
cap.set(cv2.CAP_PROP_FRAME_WIDTH,160) #设置宽度
cap.set(cv2.CAP_PROP_FRAME_HEIGHT,120)   #设置高度
numberconter=0 #数字计数
digit_copy=0 #数字副本
display(wid) #显示窗口
while True: #循环
    ret,frame=cap.read() #读取视频帧
    rois,edges,area = findRoi(frame, 50,kernel) #找到数字区域
    if rois:  #如果找到数字区域
        x, y, w, h=rois[0] #获取数字区域的坐标
        x_small=int(x+(w-h)/2) #计算小的x坐标
        if  x_small<0: 
             x_small = 0
        digit,th = findDigit(cnn,edges[y:y+h, x_small: x_small+h], 20) #识别数字
        if digit_copy==digit: #判断是否是同一个数字
            numberconter+=1 #计数加1
        if numberconter>=5: #如果计数大于5
            numberhex = digit * 10 #数字乘以10
            tCHK = (0xe0 + 0xf6 + numberhex) & 0x00ff  # 校验值
            d = struct.pack('BBBB', 0xe0, 0xf6, numberhex, tCHK) #打包数据
            # ser.write(d) #发送数据
        digit_copy=digit #保存数字
        cv2.rectangle(frame, (int(x+(w-h)/2),y), (int(x+(w+h)/2),y+h), (153,153,0), 2) #画矩形
        cv2.putText(frame, str(digit), (x,y), cv2.FONT_HERSHEY_SIMPLEX, 1, (127,0,255), 2) #写数字
       
    newEdges = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR) #转换为BGR
    newFrame = np.hstack((frame,newEdges)) #水平拼接
    wid.value = cv2.imencode('.jpg',newFrame)[1].tobytes() #显示图像

Image(value=b'')